Your name: Quang Hung Nguyen

Your student ID number: 33855953

Shared link to this notebook:https://colab.research.google.com/drive/1C-SMhLjiKVqhdUVEpj9XGvqMUbNzRjBJ?usp=sharing

## Programming Assignment 1 (P1) for COMPSCI 446, Search Engines

**WARNING**: This is a preliminary version of the P1 project. Whatever you do on this version will need to be copied over into the final version of the project. We hope that it will carry over easily.

The purpose of this project is to explore tokenization, stopword removal, and stemming within a few real documents, and to investigate term statistics as related to Heaps' and possibly Zipf's Laws. Although actual time to complete this project will vary widely across students, expect to spend several hours if you are a strong programmer and 10+ hours if you're a bit rusty.

The description below is quite lengthy and will probably feel complicated at first, but each portion is just one piece of sequence of rules that you need to apply to the tokens in the file. At a high level, the program will do the following:
* It will read in the (gzip compressed) input file and break it into tokens (loosely speaking, "words") using spaces or a fancier approach.
* It will optionally remove tokens that match stopwords in a provided list.
* It will optionally stem those tokens to their root form using a simple suffix-s stemmer or the Porter stemmer.
* It will calculate some statistics and summary information about the resulting tokens.
* It will generate incremental and cumulative information about the tokens the system encounters and generates.

Here are a list of provided files (**these are provided as described but may change for the final project**) that will be loaded into your Google Drive for use in this notebook:

* _P1-train.gz_, a compressed file that you can use as input to your program.

* _PandP.gz_, which is a much larger text file for you to play with, though we will not provide any sample output. This is the text of Jane Austen's novel, _Pride and Prejudice_, downloaded from Project Gutenberg. You can see the original at https://www.gutenberg.org/ebooks/1342; what is provided below is the UTF-8 version, but with leading and trailing material removed, and with some unusual characters converted to ASCII equivalents (e.g., smart quotation marks, the British pound symbol).

* _stopwords.txt_ is a file containing the stopwords you will use in the stopping step.


Note, after gradescope autograder is configured (**it is not ready yet**!), uploading the notebook file to gradescope will show the result on sample inputs. Please make sure the notebook is compilable (not only locally to your computer, but also on gradescope).

Execute the following to connect to Google Drive (you will be prompted repeatedly for access to your Google Drive; please give it permission) and download copies of the sample files listed above. You should not need to make any modifications to the code, though if you want to use a slightly different path in Google Drive, you can modify the appropriate drive_path value. (The autograder will not use your Google Drive.)

In [ ]:
version = 2.1 # DO NOT MODIFY. If notebook does not match with autograder version, all tests would fail!!

In [ ]:
import os
import string
import gzip
import re

from collections import Counter

try:
    from google.colab import drive
    in_colab = True
except ImportError:
    in_colab = False


# You are more than welcome to code some helper functions.
# But do note that we are only grading functions that are coded in the template files.


# Connect to Google Drive and download copies of the sample files listed above.
# Please allow the access to your Google Drive or the following dataset loader will fail.
# (The autograder will not use your Google Drive.)
if in_colab:
  drive.mount("/content/drive/") ## DO NOT MODIFY THIS LINE
  data_path = "/content/drive/MyDrive/Colab Notebooks" ## CHANGE TO YOUR OWN FOLDER ON GOOGLE DRIVE
else:
  data_path = "./data/"  ## DO NOT MODIFY THIS LINE. CHANGING THIS LINE WOULD RESULT IN FAIL OF AUTOGRADER TESTS

Mounted at /content/drive/


### 0. Pre-processing

We will need to load the texts that stored at <em>documents_path</em> before we want to perform any kind of operation on them.

Note, in this assignment, to practice tokenization process, we load the entire file directly into the memory and then process it line by line. However, in the real-world scenario, when dealing with large dataset (saying billions of entries), it is better to process the content line by line to save the space and memory of the machine.

In [ ]:
import urllib.request
import gzip
from pathlib import Path

def load_file(file_path: str, gz_zip: bool = True) -> list[str]:
    """
    Load strings from the text or gzip file. Remember to strip newline if necessary!

    Args:
        file_path: the location of file we want to analyze on.

    Returns: an array of string loaded from the text or gzip file.
    """
    webloc = "https://cs.umass.edu/~allan/cs446/"

    local_google_drive_path = os.path.join(data_path,file_path)
    local_file = Path(local_google_drive_path)
    if local_file.is_file():
        print(f"File \"{file_path}\" already exists, not downloading.")
    else:
        print(f"Cannot find \"{file_path}\" so downloading it")
        urllib.request.urlretrieve(webloc + file_path, local_google_drive_path)
        print("Done")

    if not gz_zip:
        f = open(local_google_drive_path, "r", encoding="utf-8-sig")
    else:
        # Read compressed file (opened in text mode since that's what we use)
        f = gzip.open(local_google_drive_path, "rt", encoding="utf-8-sig")
    results = [line.strip("\n") for line in f.readlines() if line]

    if f:
      f.close()

    return results



# path to documents and stopwords
documents_path = "PandP.gz" # path to gzip file that contains documents
train_data_path = "P1-train.gz" # Path to p1 train dataset
stopwords_path = "stopwords.txt"  # path to the file that contains stopwords

sentences = load_file(documents_path)
train_sentences = load_file(train_data_path)
stopwords = load_file(stopwords_path, gz_zip = False)
print(f"First line of {documents_path:>16}: {sentences[0]}");
print(f"First line of {train_data_path:>16}: {train_sentences[0]}");
print(f"First line of {stopwords_path:>16}: {stopwords[0]}");

NameError: name 'os' is not defined

### 1. Tokenization

The first step of processing text is to break it into tokens. That will be either by using whitespace or two different fancier process. How you do that will be determined by the choice of tokenization function as follows.

Each of these functions accepts a string (in this case, a line/sentence from one of the input files) and will produce a In either case, the tokens should preserve their order within original documents and if extra tokens are generated, they should be in the corresponding position of the token that triggered their creation.

#### 1.1 <u>tokenize_space()</u>

This tokenizer will break the file into a list of whitespace-separated tokens. If there are multiple whitespace characters in a row (multiple spaces, a space and a tab, a space and a line break, etc.) they are treated as a single token break: e.g., "a&lt;space&gt;&lt;space&gt;b" is two tokens ("a" and "b") rather than three ("a", &lt;empty&gt;, "b"). Do not make any other changes to the tokens beyond separating them. So, for example, the start of this paragraph would result in the following 16 tokens (for the sake of space, listed across the line separated by ⍟, though that is not what you will actually do in your output):

    This ⍟ tokenizer ⍟ will ⍟ break ⍟ the ⍟ file ⍟ into ⍟ a ⍟ list ⍟ of ⍟ whitespace-separated ⍟ tokens.

Note that punctuation is included with a token based on where the whitespace is and tokens are produced without changing their case. There are very simple solutions to this: you can do this in one line.

In [ ]:
def tokenize_space(input_str: str) -> list[str]:
    """
    Tokenize sentence into whitespace-separated tokens. For
    example, input_str = "2024 Fall Search Engine", the function
    should return ["2024", "Fall", "Search", "Engine"].

    Args:
        input_str: the sentence/phrase that we want to tokenize.

    Returns: an array of whitespace-separated tokens.
    """

    #########
    ##
    ## Implement the function here
    ##
    #########
    ans =input_str.split(' ')
    return ans# You will return something before this statement

tokens_seperated_by_space = tokenize_space(sentences[0])
print(tokens_seperated_by_space)

#### 1.2 tokenize_4grams()

This is another straightforward tokenizer, but one that does have as many ready solutions in hand. Unlike the one above, this one treats _spaces as part of the token_. Every four characters is a token, regardless of whether the characters are whitespace, punctuation, or alphanumeric characters. The one exception so that is that you should treat any white-space type character (newline or tab) as if it were a space. _However_, if there are multiple whitespace characters in a row (multiple spaces, a space and a tab, a space and a line break, etc.) they are treated as a single space: e.g., "a&lt;space&gt;&lt;space\>bc" is a single token ("a&lt;space&gt;bc") rather than the token "a&lt;space&gt;&lt;space&gt;b" and so on). Similarly, "a&lt;space&gt;&lt;newline&gt;&lt;tab&gt;bcef" should generate "a&lt;space&gt;bc" and "bcef".  

It turns out that in most applications we need to use "overlapping" n-grams. So you should start each token two characters after the previous one. Using the same notation as above, the sequence

    An  iced coffee \t
    is very nice

would generate the following sequence of tokens (using the same notation as above, with ⍟ showing where tokens break). Spaces are replaced with underscores so they are easier to see:

    An_i ⍟ _ice ⍟ ced_ ⍟ d_co ⍟ coff ⍟ ffee ⍟ ee_i ⍟ _is_ ⍟ s_ve ⍟ very ⍟ ry_n ⍟ _nic ⍟ ice

For the final token, do not include the 1-, 2-gram that ends that final token. And if the number of characters in the file is not a multiple of four, stop with the final 3-gram (like what we have in example).

In special cases, if the string is too short to create a 4-gram, you should return the original token instead.

In [ ]:
def tokenize_4grams(input_str: str) -> list[str]:
    """
    Tokenize sentence into 4char tokens with shifting windows. For example,
    input_str = "An\ticed coffee \t\nis very nice", the function should return
    ['An i', ' ice', 'ced ', 'd co', 'coff', 'ffee', 'ee i', ' is ', 's ve', 'very', 'ry n', ' nic', 'ice'].

    Args:
        input_str: the sentence/phrase that we want to tokenize.

    Returns: an array of 4-grams tokens.
    """

    #########
    ##
    ## Implement the function here
    ##
    #########
    ans =[]
    for i in range(len(input_str)-3):
        ans.append(input_str[i:i+4])
    return []  # You will return something before this statement

token_4grams = tokenize_4grams(sentences[0])

# Check if the output matches the example mentioned above
print(' ⍟ '.join([elem.replace(' ', '_') for elem in tokenize_4grams("An\ticed coffee \t\nis very nice")]))

NameError: name 'sentences' is not defined

#### 1.2 <u>tokenize_fancy()</u>

This is a more complicated tokenizer that builds on the _spaces_ tokenizer (not the 4-gram version), applying the following additional rules. You can implement these rules in any way that makes sense to you as long as you honor the result of the rules as presented below, including dependencies between the rules. They are described on the assumption that you apply them as a sequence of steps in the order listed, though. Note that you will be returning the original token and its final version(s), so it might be helpful to think about how to maintain that information.

The following tokenization rules has been listed in an order that we expect you to implement. *Hint: some of the steps generate new sequences that, in turn, need to be tokenized. You will probably want to think about a way to use recursion to handle that.*

1. Start with **ONE** space-separated token as in the <em>spaces </em>tokenizer above.

1. A token that is a URL of the form "https://&hellip;" or "http://&hellip;" should be recognized as a single token. If there is trailing punctuation, it is not part of the URL and that punctuation should be discarded. No other changes to a URL token should happen &ndash; i.e., for a URL, ignore the remaining rules below. Note that "https://hithere.com+https://howdy.org" should be considered as one simple token (since it starts with https). Do make sure to handle the cases of "HTTP" and "HTTPS" as the header of url is case-insensitive.

3. Convert the tokens to lowercase.
  
4. Treat numbers as a single token, including plus and/or minus signs, commas, and decimal points if they exist, so that "3,141.59" remains that string. A number can only be a sequence of numeric characters and those punctuation marks indicated <span class="" style="color: #98ca3e;">but it must have at least one number character. You do not have to validate that the number is well formed (so "3+14..1,59" would still be a number token). Do no other processing to number tokens.

5. Apostrophes should be "squeezed out" so that it is as if they were never there. That means that a contraction such as "don't" would turn into "dont" and a name like "O'Brian" will become "obrian" (because it is was also lowercased in step 3).

6. Any remaining punctuation <u>other than periods and hyphens</u> should be treated as a word separator (as if it were white space: note the sequence of whitespace rules from the spaces tokenizer), except recall that punctuation in a URL or number is not a word break. So one token can generate multiple tokens: "3/11//23" will result in ["3", "11", "23"], and "3^rd" will result in ["3", "rd"], but "3.14/pi" will result in ["3.14", "pi"]. If a new token is a number (as defined above), do not process it further. So in the example, "3.14" will not be treated as an abbreviation by step 8. You may prefer to do this recursively to handle these rules on sub-parts. So that "b.c.," (note the comma) would result in "b.c." (remember that periods are not word breaks) and an empty token. A recursive call on "b.c." will result in "bc" from abbreviation rule 8 and the empty token will be discarded.

7. Hyphens (not treated as word separators) within a token should be processed in three steps: (1) remove the original token with its hyphen(s), (2) add new tokens with the hyphens treated as space separators (as in the spaces tokenizer), and then (3) add a new token with all hyphens squeezed out. So,

  ["data-base"] &rarr; ["data", "base", "database"]<br/>
  ["mother-in-law"] &rarr; ["mother", "in", "law", "motherinlaw"]<br/>
  ["a.-b.-c."] &rarr; ["a.", "b.", "c.", "a.b.c."]

  Note that as with the punctuation rule, the hyphen processing will result in additional tokens. Apply all tokenization rules to each of the tokens that are generated. So "1.-on-1." will generate ["1.", "on", "1.", "1.on.1."]. The occurrences of "1." will remain unchanged because they are numbers, but "1.on.1." will be treated as an abbreviation and converted into "1on1". There are some edge cases that will cause odd things to happen (e.g., the token "1-https://Some.Url" will end up converting the embedded URL to lowercase which contractadicts rule 2), but we won't stress about unusual subtokens caused by hyphens.

8. Treat abbreviations as a single token. If a token is not a number or URL and the only punctuation it contains is periods, it is an abbreviation. Note that this includes a token that comprises nothing but periods, even if that is not intuitive. For an abbreviation, remove all periods. So "i.b.m." converts to "ibm" and "Ph.D." and "Ph.D" both go to "phd" (because an abbreviation does not need to end with a period and because rule 3 converted it to lowercase). If the result of removing the periods is an empty string, then treat it as an empty token and do not report it out.

Here are some sample tokenizations that should happen:
* Token &rarr; token (lower case rule)
* She's &rarr; shes (apostrophe rule and lower case rule)
* Mother's-IN-Law &rarr; mothers, in, law, mothersinlaw (apostrophe, lowercase, hyphens)
* U.mass &rarr; umass (lowercase, abbreviation)
* go!!!!team &rarr; go team (non-period punctuation rule)
* USD\$10.30 &rarr; usd 10.30 (non-period punctuation, case, number)
* USD\$10,30 &rarr; usd 10 30 (not period punctuation, case)
* USD\$10-30 &rarr; usd 10-30 (not-period punctuation, number (not hyphen!!))

There will be interactions between rules, and that is OK. For example,

* ["a.-2./c."] &rarr; ["a.-2.", "c."] by non-period and non-hyphen punctuation rule
* &rarr; ["a.", "2.", "a.2.", "c."] by hyphen rule
* &rarr; ["a", "2.", "a2", "c"] after applying the abbreviation rule, noting that "2." is a number so retains its decimal point.

You may use a regular expression library to help identify patterns, but other third party libraries are _not_ allowed for this part. You may, of course, use the standard Python libraries.

In [ ]:
def tokenize_fancy(input_token: str) -> list[str]:
    """
    Tokenize ONE token into tokens using the fancy rules defined above.

    Args:
        input_token: the token that we want to tokenize.

    Returns: an array of array of tokens in format,
        [
            sub_token_1_1,
            sub_token_1_2,
            ...
        ], or
        [
            'c',
            'a',
            '2.',
            'a2'
        ].
    """

    #########
    ##
    ## Implement the function here
    ##
    #########
    inp =input_token.lower().split()
    return [] # You will return something before this statement


sample_sentence = 'a.-2./c. Ph.D. Ph. D. \'\'https://hithere.com+https://howdy.org!? https://noheading.com'
# Check if the output matches the example mentioned above
for token in tokenize_space(sample_sentence):
  tokens_from_fancy_tokenization = tokenize_fancy(token)
  print(token, tokens_from_fancy_tokenization)


Sample Result:

| Phrase | Tokens |
| :------ | :------ |
| a.-2./c. | ['c', 'a', '2.', 'a2'] |
| Ph.D. | ['phd' ] |
| Ph. | ['ph'] |
| D. | ['d'] |
| \'\'https://hithere.com+https://howdy.org!? | ['https', 'hitherecom', 'https', 'howdyorg'] |
| https://noheading.com | ['https://noheading.com'] |

### 2. Stopping
After tokenizing the input file, you will apply a stopword list or not:

#### 2.1 <u>stopping()</u>
<u><strong>stopwords=None</strong></u>. means that you should not apply any stopping to the list. The result of this choice means that the token list will be unchanged by the stopping step.


<u><strong>stopwords=list[str]</strong></u>. means that you will use the list of words (that was loaded from the stopwords.txt file for you) as stopwords.

It doesn't appy in your coding of this function, but note that if an original token results in multiple tokens (spaces or non-period punctuation rules), you must apply the stopword list to <em>each of</em> the resulting tokens.

Also note that something that does not look like a stopword in the original text could become one &ndash; consider "a-n" in the text that will turn into ["a", "n", "an"] by the hyphen rule that will result in having "a" and "an" removed (if they're in the stopword list), but the "n" will be retained.

Those issues should be taken care of in the higher-level user of this function.

In [ ]:
def stopping(input_tokens: list[str], stopwords: list[str] = None) -> list[str]:
    """
    Applying stopping to the list of tokens.

    Args:
        input_tokens: the list of tokens that we want to apply stopwords.
        stopwords: the list of stopwords that need to be removed from list of token.
            Default to empty, i.e., do not stop.

    Returns: an array of array of tokens in format,
        [
            sub_token_1_1,
            sub_token_1_2,
            ...
        ], or
        [
            "a.-2./c.",
            "a",
            "2.",
            "c",
            "a2",
            "c"
        ].
    """

    #########
    ##
    ## Implement the function here
    ##
    #########
    if stopwords == None:
      return input_tokens
    else:
      for token in input_tokens:
        if token in stopwords:
          input_tokens.remove(token)
      return input_tokens
     # You will return something before this statement


sample_sentence = 'a.-2./c. and hi'
for token in tokenize_space(sample_sentence):
    stop = stopping(
        tokenize_fancy(token), stopwords=stopwords
    )
    print(token, stop)

Sample Result:

| Phrase | Tokens | Token After Stopping |
| :------ | :------ | :------ |
| a.-2./c. | ['c', '2.', 'a', 'a2'] | ['c', '2.', 'a2'] |
| and | ['and'] | [] |
| hi | ['hi'] | ['hi'] |

### 3. Stemming

After tokenizing the input file and possibly removing stopwords, you will stem each of the tokens in turn using one of these options. Note that stemming operates on every token that is not stopped, though in many cases (e.g., URLs, numbers, tokens that are already their stem) nothing will change. Also note that if a token generated multiple tokens (the spaces or non-period punctuation rules), the stemmer must be applied to each of them.

#### 3.1 Stemming "s" suffix
<u><strong>stemming_type=None</strong></u>. means that you apply no stemming, so the token passes unchanged through this step.

<u><strong>stemming_type=suffix_s</strong></u>. means that you apply the "suffix-s" stemmer, where if the final character of a token is "s" you remove the "s" and otherwise leave the token unchanged.

(Porter comes next)

In [ ]:
def stemming_s(input_tokens: list[str]) -> list[str]:
    """
    Applying suffix-s stemming to the list of tokens.

    Args:
        input_tokens: the list of tokens that we want to perform stemming.

    Returns: an array of array of tokens in format,
        [
            sub_token_1_1,
            sub_token_1_2,
            ...
        ]

        For example, if the input is ["hi", "is", "this", "lass", "silly"] the output should be:
        [ "hi", "i", "thi", "las", "silly" ]
    """

    #########
    ##
    ## Implement the function here
    ##
    #########
    for token in input_tokens:
      if token[-1] == 's':
        input_tokens.remove(token)
        input_tokens.append(token[:-1])
    return [] # You will return something before this statement


sample_sentence = 'hi is this lass silly'
for token in tokenize_space(sample_sentence):
    stem = stemming_s(tokenize_fancy(token))  # Just tokenizes and skips stopping
    print(token, stem)

Sample Result:

| Phrase | Tokens |
| :------ | :------ |
| hi | ['hi'] |
| is | ['i'] |
| this | ['thi'] |
| lass | ['las'] |
| silly | ['silly'] |

#### 3.2 Stemming with Porter Stemmer

<u><strong>stemming_type=porter</strong></u>. means that you apply the Porter stemmer (we will use "the English stemmer" which is also called "Porter2"), though you will only implement part of the stemming algorithm. You'll find a description of the Porter2 stemmer in your textbook with Steps 1a and 1c listed. However, please use the following restatement of the algorithm where some of the ambiguities are clarified and where Step 1c is included.

For each token in turn, apply the rule of Step 1a that matches the longest suffix (if it matches any) and then apply the one rule of Step 1b that matches the longest suffix (if any) of the output of Step 1a and then apply Step1c to Step 1b's result if it fits. In each step, if no suffix matches, do nothing and then continue to the next step.

    Vowels are a, e, i, o, and u.

As discussed in class (on Tuesday, September 17), the algorithm in the book talks about several things that are ill-defined as is. To address them, you need a method to determine a type of "structure" of the token being stemmed. Here that means figuring out how the token fits the pattern $[C](VC)^m[V]$ where $V$ is a sequence of one or more vowels (as above), C is a sequence of one or more consonents (anything that is not a vowel is a consonent), and $m$ indicates how often the $CV$ pattern repeats: if $m=0$ then it does not occur at all. Note that every word can be represented by this pattern. (Note that it's a weird concept for the 4-gram tokenizer. Treat the space as if it is a consonent.)

Here are some examples to help you verify you're doing the right thing:

* _agreed_ is _VCVC_ which is $(VC)^2$ with the leading $C$ and trailing $V$ empty.
* _retrieval_ is _CVCVCVC_ which is $C(VC)^3$ with the trailing $V$ empty.
* _feed_ is _CVC_ which is $C(VC)^1$ with the trailing $V$ empty.
* _tree_ is _CV_ which is $C(VC)^0V$ or $CV$ with nothing in the middle.

And now, on to the steps of the algorithm, reworked slightly from the description in the book for clarity. Recall that each step is looking at the _end_ of the word.

Step 1a:
  * Replace <em>sses</em> by <em>ss</em> (e.g., <em>stresses&rarr;stress</em>) and do nothing else for Step 1a.
  * If the stem ends with <em>ied</em> or <em>ies</em>, then if the remaining stem is more than one character long, replace the ending by _i_, otherwise replace it by _ie_ (e.g., <em>ties&rarr;tie, cries&rarr;cri</em>). In either case, do nothing else for Step 1a.
  * If the stem ends in <em>us </em>or <em>ss </em>do nothing (e.g., <em>stress&rarr;stress</em>), including doing nothing else for Step 1a
  * If the stem ends with _s_, then "if the preceding stem part contains a vowel not immediately before the _s_" then delete the trailing _s_  (e.g., <em>gaps&rarr;gap</em> but <em>gas&rarr;gas</em>). And do nothing else for Step 1a. (Note that the "contains a vowel" bit is not helped by the $m$ rule described above. You'll need another way to check for that.)

Step 1b:
  * If the stem ends in <em>eed </em>or <em>eedly </em><u>then:</u>
      * if it is in the part of the stem after the first non-vowel following a vowel (i.e., if $m>1$ in the token that gets to 1b), replace it by <em>ee </em>(e.g., <em>agreed&rarr;agree, feed&rarr;feed</em>).
      * then go to step 1c
  * If the stem ends in <em>ed, edly, ing</em>, or <em>ingly </em>then:<u><strong> <br /></strong></u>
      * <u><strong></strong></u>if the preceding stem part <em>does not</em> contain a vowel, go to step 1c

      * if the preceding stem part <em>does</em> contain a vowel delete the ending <u><em>and then also consider the following three possibilities with the resulting stem:</em></u>
          * if the stem now ends in <em>at</em>, <em>bl</em>, or <em>iz </em>add _e_ (e.g., <em>fished &rarr; fish, pirating &rarr;pirate</em>) and go to step 1c

          * if the stem now ends with a double letter that is one of <em>bb, dd, ff, gg, mm, nn, pp, rr,</em> or <em>tt</em>, remove the last letter (e.g., <em>falling&rarr;fall, dripping&rarr;drip</em>) and go to step 1c

          * if the stem is now short (defined as $m=1$ for the stem after removing the suffix), add _e_ (e.g., <em>hoping&rarr;hope</em>) and go to step 1c.
             * Some example short words by this definition are <em>at </em>or <em>ow </em>(or "<em>or</em>"!). Surprisingly, <em>be</em> is not short because it is $CV = C(VC)^0V$, so $m=0$ which is not 1.
             * These stems are also short: <em>shed, shred, rap, trap, hop</em>, or <em>bid </em>, yet these are not: <em>box</em>, <em>bow</em>, or <em>beds</em>).

Step 1c (not in the textbook):
  * If the stem ends in <em>y </em>and the character before the <em>y </em>is neither a vowel nor the first letter of the word, replace the <em>y </em>with an <em>i </em>(e.g., <em>cry&rarr;cri, bi&rarr;bi, bambi&rarr;bamby, say&rarr;say</em>)

Note: stemming_type matching should be case-insensitive, i.e., stemming_type="PoRtEr" should also return the tokens after applying porter stemmer. Also, the default setting of <u>stemming()</u> should always be return the original tokens without stemming.

In [ ]:
def stemming(input_tokens: list[str], stemming_type: str = None) -> list[str]:
    """
    Applying stemming to the list of tokens.

    Args:
        input_tokens: the list of tokens that we want to apply stopping list.
        stemming_type: (case-insensitive) the stemming method that we want to apply on the token list.
              Options are [None, "suffix_s", "porter"].
              Default and unrecognized value(s) are considered as None.

    Returns: an array of array of tokens in format,
        [
            sub_token_1_1,
            sub_token_1_2,
            ...
        ], or
        [
            "a.-2./c.",
            "2.",
            "c",
            "a2",
            "c"
        ].
    """

    #########
    ##
    ## Implement the function here
    ## Make sure also include the suffix_s stemming function and return the results accordingly
    ##
    #########
    if stemming_type == "suffix_s":
      return []
    elif stemming_type == "porter":
      return []
    else:

    return [] # You will return something before this statement


sample_sentence = 'a.-2./c. pirating gas hoping'
for token in tokenize_space(sample_sentence):
    stem = stemming_porter(
        stopping(tokenize_fancy(token), stopwords=stopwords),
        stemming_type="porter"
    )
    print(token, stem)

Sample Result:

| Phrase | Tokens | Tokens After Stopping and Stemming |
| :------ | :------ | :------ |
| a.-2./c. | ['c', '2.', 'a', 'a2'] | ['c', '2.', 'a2'] |
| pirating | ['pirating'] | ['pirate'] |
| gas | ['gas'] | ['gas'] |
| hoping | ['hoping'] | ['hope'] |

### 4. Output & Statistics

In the following, you will implement three function: <u>tokenization()</u>, <u>heaps()</u>, <u>statistics()</u>, which use the functions we just implemented.

#### 4.1 Tokenization with or without stopping and stemming: <u>tokenization()</u>

<u>tokenization()</u> will tokenize a list of sentences given the option of your desire and return a list of processed token in the required format. Each element of returned list (a tuple) will contain the original space-separated word and a list of token that are generated using the original space-separated word. So if fancy tokenization, normal stopping, and the Porter stemming are enabled, the sentences

    sentence 1: "whitespace-Separated" ⍟ tokens ⍟ (as ⍟ in ⍟ P0).
    sentence 2: And, ⍟ a.-2./c. ⍟ also.
    
where "⍟" indicates whitespace sequences, would result in

| Phrase | Tokens|
| :------- | :-------- |
|<span class="" style="color: #98ca3e;">"whitespace-Separated"| ["whitespace", "separate", "whitespaceseparate"] |
|<span class="" style="color: #98ca3e;">tokens| ["token"] |
|<span class="" style="color: #98ca3e;">(as| [] |
|<span class="" style="color: #98ca3e;">in| [] |
|<span class="" style="color: #98ca3e;">P0).| ["p0"] |
|<span class="" style="color: #98ca3e;">And,| [] |
|<span class="" style="color: #98ca3e;">a.-2./c.| ["2.", "c", "a2"] |
|<span class="" style="color: #98ca3e;">also.| ["also"] |


Several expectation as follows:
* The tokens must be listed in the order that the original tokens were encountered in the input file.
* If an original token is removed (e.g., stopped) nothing will be printed for that token (e.g., the line for <span class="" style="color: #98ca3e;">"(as", <span class="" style="color: #98ca3e;">"in", and <span class="" style="color: #98ca3e;">"And,").
* If an original token is transformed into additional tokens (e.g., because of hyphens) all of the new tokens will be listed on the same line (e.g., the hyphenated word and the weird <span class="" style="color: #98ca3e;">"a.-2./c." sequence).
* If a token is stemmed, the stem will be displayed with the original token. Note that the hyphenated word is separated, made into three tokens, and each of them is stemmed.
* Note that the process of tokenization with stopping and stemming can be executed as you read through the input file, though in this assignment we choose to read all the content in the file and then process it.

In [ ]:
def tokenization(
    input_list_str: list[str], stopwords: list[str] = None, stemming_type: str = None
) -> list[tuple[str, list[str]]]:
    """
    Tokenize the input sentences and apply stopping and stemming procedure to the tokenized sentences.

    Args:
        input_list_str: the list of sentences that we read from the input file.
        stopwords: the list of stopwords that need to be removed from list of token.
        stemming_type: (case-insensitive) the stemming method that we want to apply on the token list.
              Options are [None, "suffix_s", "porter"].
              Default and unrecognized value(s) are considered as None.

    Returns: an array of processed tokens in required format, i.e.,
        [
            ("\"whitespace-Separate\"", ["whitespace", "separate", "whitespaceseparate"]),
            ("Tokens", ["token"]),
            ...
        ].
    """

    #########
    ##
    ## Implement the function here
    ##
    #########

    return []  # You will return something before this statement


sample_sentences = ['a.-2./c. pirating', 'gas hoping']
processed_tokens = tokenization(sample_sentences, stopwords=stopwords, stemming_type="porter")
for phrase, tokens in processed_tokens:
    print(phrase, tokens)

Sample Result:

| Phrase | Tokens |
| :------ | :------ |
| a.-2./c. | ['c', '2.', 'a2'] |
| pirating | ['pirate'] |
| gas | ['gas'] |
| hoping | ['hope'] |

#### 4.2 Token Statistics - <u>heaps()</u>
<u>heaps()</u> analyze the processed tokens and count the number of valid tokens.

Do not count stopwords if they are being removed, but do count the extra words that hyphens and other non-period punctuations generate. The return should have the format:[ (10, 8), (20, 17), ... ], where the first compoenent is the number of tokens you have generated so far (will be increasing multiples of 10, except possibly the very last one) and the second compoenent is the number of those tokens so far if you remove duplicates. For both counts, these are the tokens after the process of tokenizing, stopping, and stemming steps (indicated by the option in <u>tokenization()</u>), not before those steps. Clearly the second count will never be greater than the first.

In [ ]:
def heaps(processed_tokens: list[tuple[str, list[str]]]) -> list[tuple[int, int]]:
    """
    Analyze the processed tokens.

    Args:
        processed_tokens: the list of processed tokens in the expected format.

    Returns: an array of statistics in required format, i.e.,
        [
            (10, 8),
            (20, 17),
            ...
        ].
    """

    #########
    ##
    ## Implement the function here
    ##
    #########

    return []  # You will return something before this statement

processed_tokens = tokenization(sentences, stopwords=stopwords, stemming_type="porter")
heaps_result = heaps(processed_tokens)
[print(curr) for curr in heaps_result[:10]];

#### 4.3 Token Statistics - <u>statistics()</u>

<u>statistics()</u> analyze the statistic of the processed tokens and return the following information in this order:
    <ul>
        <li>A line containing the total number of tokens you encountered</li>
        <li>A line containing the total number of unique tokens you encountered</li>
        <li>An array contains the 100 most frequent final tokens you generated &ndash; i.e., after all of tokenizing, stopping, and stemming are done. Each element should have the format <code>[Token, Token_num]</code> where Token_num is the number of occurrences of token you found. <span class="" style="color: #98ca3e;">Sort them in descending order by the value of Token_num. If there are multiple words with the same value of Token_num, sort them alphabetically by the token. <span class="" style="color: #98ca3e;">If the same value of Token_num happens for the 100th and 101st items, just stop at 100.</li>
    </ul>

Note that the first total number of tokens and unique tokens should match with the last element of the result of <u>heaps()</u>.
</div>

In [ ]:
def statistics(
    processed_tokens: list[tuple[str, list[str]]]
) -> tuple[int, int, list[tuple[str, int]]]:
    """
    Analyze the processed tokens.

    Args:
        processed_tokens: the list of processed tokens in the expected format.

    Returns:
        total number of tokens encountered;
        total number of unique tokens encountered;
        100 most frequent processed tokens encountered in required format, i.e.,
        [
            ("Test", 10),
            ("Token", 10),
            ("Apple", 9),
            ...
        ].
    """

    #########
    ##
    ## Implement the function here
    ##
    #########

    return []  # You will return something before this statement

processed_tokens = tokenization(sentences, stopwords=stopwords, stemming_type="porter")
stats_result = statistics(processed_tokens)
print(f"Overall Token Counts: {stats_result[0]}; Overall Unique Token Counts: {stats_result[1]}")
print("Top 10 frequent tokens with count:")
[print(curr) for curr in stats_result[2][:10]];

### 5. Analysis questions

#### 5.1
Run your program with fancy tokenization, stopping, and Porter stemming on <em>PandP.gz</em> and look at the statistics() output to see the most frequent terms from _Pride and Prejudice_. Print the most frequent 100 words and their counts out, sorted in descending order by count). Break ties around the 100th word by alphabetical order of the tokens.

After the list of terms, explain how the top terms are or are not relevant to the story or whether they seem like everyday words that aren't particularly to the novel? Support your answer with examples. You may find it useful to skim the <a class="" href="https://en.wikipedia.org/wiki/Pride_and_Prejudice">summary on Wikipedia</a> to know what words are part of the story.

    Enter your answer here...

#### 5.2
For this question, copy those same 100 words. Then say whether any of those top terms should have been stopwords (in your opinion)? Do the top terms or the list of all tokens suggest any changes to the tokenizing or stemming rules? What are they and why should they be made?

    Enter your answer here...

#### 5.3 Graph Comparison between result and Heaps' law - <u>graph_comparison()</u>

Figure 4.4. in the textbook (p. 82) displays a graph of vocabulary growth for the TREC GOV2 collection. <u>graph_comparison()</u> should generate a similar graph which comparing the difference between the results of processed documents (from <u>heaps()</u>) and that of Heaps' Law. Try out some _K_ and _B_ values such that the produced line approximates the line comes from the dataset.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def graph_comparison(K: float, B: float) -> None:
    """
    Generates a graph that compares the results of processed documents and that of heaps' law.
    The curvature of heaps' law is computed using the function f(x) = K*x^B.

    Args:
        K: heaps' law parameter,
        B: heaps' law parameter.
    """

    heaps = np.array(heaps_result)

    X = heaps[:,0]
    dataset_y = heaps[:,1]
    heaps_y = [K * (elem ** B) for elem in X]

    _, ax = plt.subplots(1)
    ax.plot(X, heaps_y, "--", label=f"Heaps {K} {B}")
    ax.plot(X, dataset_y, label="Dataset")
    ax.set_title("Vocabulary growth for the Dataset")
    ax.set_xlabel("Words in Collection")
    ax.set_ylabel("Words in Vocabulary")
    ax.set_xlim(xmin=0)
    ax.set_ylim(ymin=0)
    plt.legend()
    plt.show()


K = 40
B = 0.5
graph_comparison(K=K, B=B)

NameError: name 'heaps_result' is not defined

(5.3 continued)

Consider the graph generated above. Then say whether you feel that _Pride and Prejudice_ follows **Heaps' Law**? Why do you think so or think not? How did you decide on values of B and K?

Note: when selecting pages for the question, make sure to include the graph along with your answer.

    Enter your answer here


### 6. Extra credit

The following is an extra credit option for this assignment. Success on this part will possibly increase your score on P1. If you submit extra credit that does not work at all, you will lose one point on P1! **NOTE** We haven't figured out how much this is worth yet.



In [ ]:
# Copy graph_comparison's code here to create one for zipf that plots the passed data and calculates the "ideal" Zipf's curve for comparison

def zipf_graph_comparison(c: float = None) -> None:
    """
    Generates a graph that compares the results of processed documents and that of zipf's law.
    The curvature of zipf's law is computed using the function f(x) = c/x, where x is the rank.

    Args:
        c: zipf's law parameter. Using rank 1's probability if not specified.
    """

zipf_graph_comparison()


Consider the graph generated above. Then say whether you feel that _Pride and Prejudice_ follows **Zipf's law**? Why do you think so or think not? Discuss your idea with some reasoning.

Note: when selecting pages for the question, make sure to include the graph along with your answer.

    Please enter your analysis of Zipf's Law here

### 7. Misc & Grading

When you submit your code, the autograder will run:

* up to 18 possible combinations of the command line on the training input (3 tokenizers, 2 stemmers + not stemming, stopping or not)

* at least spaces-noStop-noStem and fancy-yesStop-porterStem on _Pride and Prejudice_

* at least fancy-yesStop-porterStem on a held-out evaluation set that you do not have

You may find a utility such as <em>diff</em> (Linux) helpful to compare your output to the expected output on <em>P1-train</em>. For the others, you'll have to eyeball it to see if it looks right. Or you may want to write a quick comparison function for your own use.

Some things that you might want to consider while coding and debugging:

* If you start with the <em>spaces</em> tokenizer and turn off stopping and stemming, you can check that this foundation step (breaking the text into space-separated tokens) works. That happens to be the same as the spaces-noStop-noStem sample output for P1-train.

* Running the output of spaces though the stopping and stemming algorithms isn't likely to be very helpful for you (though we will grade it!) because the leftover punctuation reduces the number of times stemming occurs.

* You can compare spaces-noStop-noStem to fancy-noStop-noStem to see if your fancier tokenizer is working. You can look to see that numbers and URLs pass through unchanged, the hyphens and other punctuation behave as described, and so on.

* Then shift to fancy-yesStop-noStem to see if stopping is working

* And then fancy-yesStop-yesStem for see if the whole process works